In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

risk_free_rate = 0.03
trial_n = 3

tickers = ['NKE', 'MSFT', 'DIS', 'COST']

start = '2012-1-1'
end = '2022-3-1'

df_close = yf.download(tickers, start, end, interval='1mo')['Close']

df_close.to_csv('./data/df_close.csv')

[*********************100%***********************]  4 of 4 completed


In [2]:
df_close.shape

(122, 4)

In [3]:
df_close.head()

,COST,DIS,MSFT,NKE
Date,,,,
2012-01-01 00:00:00-05:00,82.269997,38.900002,29.530001,25.997499
2012-02-01 00:00:00-05:00,86.059998,41.990002,31.740000,26.980000
2012-03-01 00:00:00-05:00,90.800003,43.779999,32.259998,27.110001
2012-04-01 00:00:00-04:00,88.180000,43.110001,32.020000,27.967501
2012-05-01 00:00:00-04:00,86.389999,45.709999,29.190001,27.045000


In [4]:
df_close.tail()

,COST,DIS,MSFT,NKE
Date,,,,
2021-10-01 00:00:00-04:00,491.540009,169.070007,331.619995,167.289993
2021-11-01 00:00:00-04:00,539.380005,144.899994,330.589996,169.240005
2021-12-01 00:00:00-05:00,567.700012,154.889999,336.320007,166.669998
2022-01-01 00:00:00-05:00,505.130005,142.970001,310.980011,148.070007
2022-02-01 00:00:00-05:00,519.250000,148.460007,298.790009,136.550003


In [5]:
df_close.isnull().sum()

COST    0
DIS     0
MSFT    0
NKE     0
dtype: int64

In [6]:
if df_close.isnull().values.any():
    df_close.fillna(method='ffill', inplace = True) #front fills
    df_close.fillna(method='bfill', inplace = True) #back fills

In [7]:
df_pct_change = df_close.pct_change().iloc[1:] #get rid of last row because cannot calc pct change for index +1
df_pct_change.head()

,COST,DIS,MSFT,NKE
Date,,,,
2012-02-01 00:00:00-05:00,0.046068,0.079434,0.074839,0.037792
2012-03-01 00:00:00-05:00,0.055078,0.042629,0.016383,0.004818
2012-04-01 00:00:00-04:00,-0.028855,-0.015304,-0.007439,0.031630
2012-05-01 00:00:00-04:00,-0.020299,0.060311,-0.088382,-0.032985
2012-06-01 00:00:00-04:00,0.099664,0.061037,0.047962,-0.188575


In [8]:
mean_return = df_pct_change.mean() * 12
mean_return

COST    0.198531
DIS     0.160302
MSFT    0.251882
NKE     0.190050
dtype: float64

In [9]:
var_risk = df_pct_change.var() * 12
var_risk

COST    0.029159
DIS     0.055223
MSFT    0.041740
NKE     0.049432
dtype: float64

In [10]:
cov_mat = df_pct_change.cov()*12
cov_mat

,COST,DIS,MSFT,NKE
COST,0.029159,0.011473,0.011932,0.012717
DIS,0.011473,0.055223,0.017132,0.020052
MSFT,0.011932,0.017132,0.041740,0.013990
NKE,0.012717,0.020052,0.013990,0.049432


In [11]:
corr_mat = df_pct_change.corr()
corr_mat

,COST,DIS,MSFT,NKE
COST,1.000000,0.285910,0.342005,0.334956
DIS,0.285910,1.000000,0.356845,0.383797
MSFT,0.342005,0.356845,1.000000,0.308000
NKE,0.334956,0.383797,0.308000,1.000000


In [12]:
weight = 1/len(tickers)
eq_weights = [weight for i in tickers]
eq_weights

[0.25, 0.25, 0.25, 0.25]

In [13]:
eq_return = eq_weights @ (mean_return)
eq_return

0.20019129578488354

In [14]:
eq_risk = ((eq_weights @ cov_mat) @ eq_weights)**(1/2)
eq_risk

0.14793324244566944

In [15]:
eq_sharpe = (eq_return - risk_free_rate)/eq_risk
eq_sharpe

1.150460119519037

In [16]:
efficient_frontier_data = []
for trial in range(trial_n):
    t_weight = np.random.random(len(tickers, ))
    t_weights = t_weight/sum(t_weight)
    t_return = t_weights @ (mean_return)
    t_risk = ((t_weights @ cov_mat) @ t_weights)**(1/2)
    t_sharpe = (t_return - risk_free_rate)/t_risk
    efficient_frontier_data.append([t_weights, t_risk, t_return, t_sharpe])

efficient_frontier_data = pd.DataFrame(efficient_frontier_data, columns=['Weights', 'Risk', 'Return', 'Sharpe'])
efficient_frontier_data.head()

,Weights,Risk,Return,Sharpe
0,"[0.27714061477257074, 0.3010565731222685, 0.15...",0.150094,0.193207,1.087364
1,"[0.03817834811651114, 0.21687362663057502, 0.2...",0.164076,0.202441,1.050982
2,"[0.3971571050824094, 0.36762024472695254, 0.10...",0.150502,0.188934,1.056028


In [17]:
i = efficient_frontier_data['Sharpe'].idxmax()
max_sharpe = efficient_frontier_data.iloc[i, :]
max_sharpe

Weights    [0.27714061477257074, 0.3010565731222685, 0.15...
Risk                                                0.150094
Return                                              0.193207
Sharpe                                              1.087364
Name: 0, dtype: object

In [18]:
i = efficient_frontier_data['Risk'].idxmin()
min_risk = efficient_frontier_data.iloc[i, :]
min_risk

Weights    [0.27714061477257074, 0.3010565731222685, 0.15...
Risk                                                0.150094
Return                                              0.193207
Sharpe                                              1.087364
Name: 0, dtype: object

In [19]:
i = efficient_frontier_data['Return'].idxmax()
max_return = efficient_frontier_data.iloc[i, :]
max_return

Weights    [0.03817834811651114, 0.21687362663057502, 0.2...
Risk                                                0.164076
Return                                              0.202441
Sharpe                                              1.050982
Name: 1, dtype: object